### Домашнее задание Transformers Training (50 баллов)

В этом домашнем задании требуется обучить несколько Transformer-based моделей в задаче машинного перевода. Для обучения можно воспользоваться текущим проектом, так и реализовать свой пайплайн обучения. Если будете использовать проект, теги **TODO** проекта отмечают, какие компоненты надо реализовать.
В ноутбуке нужно только отобразить результаты обучения и выводы. Архитектура модели(количетсво слоев, размерность и тд) остается на ваш выбор.

Ваш код обучения нужно выложить на ваш github, в строке ниже дать ссылку на него. В первую очередь будут оцениваться результаты в ноутбуке, код нужен для проверки адекватности результатов. 

Обучать модели до конца не нужно, только для демонстрации, что модель обучается и рабочая - снижение val_loss, рост bleu_score.

### Данные

`
wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip
`

Модели нужно обучить на задаче перевода с английского на русский. 

#### Сcылка на ваш github с проектом(вставить свой) - https://github.com/runnerup96/pytorch-machine-translation

Ноутбук с результатами выкладывать на ваш **google диск** курса. 

In [1]:
# !wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip

In [7]:
from datamanip import get_dataset, get_tokenizer,compute_metrics, preprocess_function, DatasetConfig

ImportError: cannot import name 'get_dataset' from 'data' (/home/jovyan/runfme/triton/translate/data.py)

# Файнтюн т5
результаты: https://api.wandb.ai/links/runfme/ktxhl153

In [3]:
from omegaconf import OmegaConf
from transformers import DataCollatorForSeq2Seq

conf = OmegaConf.load('./cfg_tune_t5.yaml')
dataset_cfg = OmegaConf.structured(DatasetConfig(**conf['dataset']))
model_cfg = OmegaConf.structured(ModelConfig(**conf['model']))

tokenizer = get_tokenizer(dataset_cfg)

dataset = get_dataset(dataset_cfg)
dataset = dataset.train_test_split(test_size=0.03)
tokenized_dataset = dataset.map(preprocess_function, batched=True)

model = get_model(model_cfg)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/user/conda/envs/runfme_default/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map:   0%|    

In [10]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import wandb

import os
os.environ["WANDB_API_KEY"]= '844ae81dd9f3dac384cf5cdb478d4e939ed71fa1'
wandb.init()
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_tune-t5",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=0.0005,
    lr_scheduler_type='cosine',
    warmup_steps=200,
    per_device_train_batch_size=80,
    per_device_eval_batch_size=80,
    weight_decay=0.00,
    save_total_limit=3,
    max_steps=3000,
    report_to="wandb",
    predict_with_generate = True,
    generation_max_length=30,
    fp16=False,
    load_best_model_at_end=True,
    greater_is_better=True,
    metric_for_best_model="bleu",
    gradient_accumulation_steps=2,
    logging_steps=2,
    logging_nan_inf_filter=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/user/conda/envs/runfme_default/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu
500,1.645000,1.167628,28.093765
1000,1.358500,0.985108,34.808586
1500,1.167300,0.892684,37.879124
2000,1.101900,0.856037,39.683768
2500,1.066800,0.833254,40.442491
3000,1.101500,0.830280,40.460239


[[    0 33944   315 ...  -100  -100  -100]
 [    0 33944 76313 ...  -100  -100  -100]
 [    0  2553   401 ...  -100  -100  -100]
 ...
 [    0  5778   279 ...  -100  -100  -100]
 [    0  2553 37641 ...  -100  -100  -100]
 [    0 30326  1211 ...  -100  -100  -100]] [[33944   315 32435 ...  -100  -100  -100]
 [33944 76313 17714 ...  -100  -100  -100]
 [ 2553   401   259 ...  -100  -100  -100]
 ...
 [ 5778   279 24193 ...  -100  -100  -100]
 [14152   396 37641 ...  -100  -100  -100]
 [30326   279  3748 ...  -100  -100  -100]]
[[    0 33944   315 ...  -100  -100  -100]
 [    0 33944 76313 ...  -100  -100  -100]
 [    0  2553   401 ...  -100  -100  -100]
 ...
 [    0  5778   279 ...  -100  -100  -100]
 [    0  2553 37641 ...  -100  -100  -100]
 [    0 30326  1211 ...  -100  -100  -100]] [[33944   315 32435 ...  -100  -100  -100]
 [33944 76313 17714 ...  -100  -100  -100]
 [ 2553   401   259 ...  -100  -100  -100]
 ...
 [ 5778   279 24193 ...  -100  -100  -100]
 [14152   396 37641 ...  -100  

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=3000, training_loss=1.8637264870802561, metrics={'train_runtime': 1860.3313, 'train_samples_per_second': 258.019, 'train_steps_per_second': 1.613, 'total_flos': 9966433904824320.0, 'train_loss': 1.8637264870802561, 'epoch': 1.53})

In [16]:
model = AutoModelForSeq2SeqLM.from_pretrained('./results_tune-t5/checkpoint-3000').cuda()

for i in dataset['test'].select(range(10)):
    source, target = i['source'], i['target']
    inputs = tokenizer(
        source,
        return_tensors='pt',
        max_length=256,
        truncation=True
    )

    # Get correct sentence ids.
    corrected_ids = model.generate(
        inputs.input_ids.cuda(),
        max_length=256,
        num_beams=1, # `num_beams=1` indicated temperature sampling.
        early_stopping=True
    )

    # Decode.
    res = tokenizer.decode(
        corrected_ids[0],
        skip_special_tokens=True
    )
    print(f'Eng: {source}')
    print(f'Reference: {target}')
    print(f'Predicted: {res}')

/home/user/conda/envs/runfme_default/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Eng: We can't work without electricity.
Reference: Мы не можем работать без электричества.
Predicted: Мы не можем работать без электричества.
Eng: I didn't know Tom liked jazz.
Reference: Я не знал, что Том любит джаз.
Predicted: Я не знал, что Тому нравится рок.
Eng: I have lost the key.
Reference: Я ключ потерял.
Predicted: Я потерял ключ.
Eng: Tom had heard nothing about the accident.
Reference: Том ничего не слышал об аварии.
Predicted: Том ничего не слышал о аварии.
Eng: What's your favorite tree?
Reference: Какое у тебя любимое дерево?
Predicted: Какой у тебя любимый деревь?
Eng: Tom did this.
Reference: Том это сделал.
Predicted: Том это сделал.
Eng: Why do you love Tom?
Reference: Почему вы любите Тома?
Predicted: Почему вы любите Тома?
Eng: That's what happens when one tells the truth.
Reference: Вот что бывает, когда говоришь правду.
Predicted: Это что происходит, когда одна говорит правду.
Eng: Where did you buy these towels?
Reference: Где вы купили эти полотенца?
Predicted

### Обучение Seq2seq Transformer модель(25 баллов)

Реализуйте Seq2seq Transformer. В качестве блока трансформера можно использовать https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html. В качестве токенизатора воспользуйтесь HuggingFace токенизатор для source/target языков - https://huggingface.co/docs/transformers/fast_tokenizers
В качестве максимальной длинны возьмите предложения длинной **до 15 слов**, без каких либо префиксов. 

Не забудьте остальные элементы модели:
* Мы можем использовать 1 трансформер как энкодер - декодером будет выступать линейный слой. 
* Обучите свой BPE токенизатор - https://huggingface.co/docs/transformers/fast_tokenizers
* Матрицу эмбеддингов токенов
* Матрицу позицонных эмбеддингов
* Линейный слой проекции в target словарь
* Функцию маскирования будущих состояний attention, так как модель авто-регрессионна
* Learning rate schedualer


В качестве результатов, приложите слудующие данные:
1) Параметры обучения - learning rate, batch_size, epoch_num, размерность скрытого слоя, количетсво слоев
2) Графики обучения - train loss, val loss, bleu score
3) Примеры переводов вашей модели(10 штук) - source text, true target text, predicted target text